### Кодирование текста в качестве мешка слов
Даны текстовые данные, и требуется создать набор признаков, <br>
указывающих на количество вхождений определенного слова в текст наблюдения.

In [1]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

text_data = np.array(['Бразилия - моя любовь. Бразилия!',
                      'Швеция - лучше',
                      'Германия бьет обоих'])
count = CountVectorizer()
bag_of_words = count.fit_transform(text_data)
bag_of_words

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 8 stored elements and shape (3, 8)>

In [2]:
bag_of_words.toarray()

array([[2, 0, 0, 0, 1, 1, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 1],
       [0, 1, 1, 0, 0, 0, 1, 0]])

In [4]:
count.get_feature_names_out()

array(['бразилия', 'бьет', 'германия', 'лучше', 'любовь', 'моя', 'обоих',
       'швеция'], dtype=object)

Одним из наиболее распространенных методов преобразования текста в признаки является использование модели мешка слов.<br>
Эти модели выводят признак для каждого уникального слова в текстовых данных, при этом каждый признак содержит кол-во вхождений в наблюдениях.<br><br>
Большинство слов, не встречаются в большинстве наблюдений, и поэтому матрицы признаков на основе мешка слов будут в качестве значений содержать в основном нули. Эти матрицы называются ***разреженного*** типа. Одной из особенностей векторизатора частностей CountVectorizer <br>
 <br>
 Каждый признак является словом - это необязательно. Вместо этого мы можем установить, чтобы каждый признак был комбинацией <br>
 двух слов (2-граммами) или трех слов (3-граммами). Параметр ngram_range устанавливает минимальный и максимальный размеры наших n-gram. <br>
 Так же можно удалить слова с низкой информацией, используя стоп-слова в параметре stop_words либо с помощью встроенного списка, либо с помощью собственного списка

In [6]:
count_2gram = CountVectorizer(ngram_range=(1, 2),
                              stop_words='english',
                              vocabulary=['бразилия'])
bag = count_2gram.fit_transform(text_data)
bag.toarray()

array([[2],
       [0],
       [0]])

### Взвешивание важности слов

Требуется мешок слов, но со словами, взвешенными по их важности для наблюдения.<br>
Решение: сравнить частоту слова в документе, используя - обратную документную частоту (tf-idf).

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

text_data = np.array(['Бразилия - моя любовь. Бразилия!',
                      'Швеция - лучше',
                      'Германия бьет обоих'])
vectorizer = TfidfVectorizer() # приводит токены к нижнему регистру  и удаляет пунктуацию по умолчанию
tfidf_matrix = vectorizer.fit_transform(text_data)
tfidf_matrix.toarray()

array([[0.81649658, 0.        , 0.        , 0.        , 0.40824829,
        0.40824829, 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.70710678, 0.        ,
        0.        , 0.        , 0.70710678],
       [0.        , 0.57735027, 0.57735027, 0.        , 0.        ,
        0.        , 0.57735027, 0.        ]])

In [19]:
vectorizer.vocabulary_

{'бразилия': 0,
 'моя': 5,
 'любовь': 4,
 'швеция': 7,
 'лучше': 3,
 'германия': 2,
 'бьет': 1,
 'обоих': 6}

Чем чаще слово попадается в документе, тем более вероятно, что оно важно для этого документа.<br>
Эта мера частоты встречаемости слова в документе называется словарной частотой (tf).<br><br>
А если слово встречается во многих документах, но менее важно для любого отдельного документа, <br>
то эта мера называется документной частотой (df).<br><br>
Объединив эти два статистических показателя, мы можем назначить оценку каждому слову, тем самым показывая, насколько важно это слово:
$$
tf-idf(t,d)=tf(t,d)\times idf(t,d)
$$
где *t* - слово; *d* - документ.<br>
tf - это просто количество раз, когда слово появляется в документе, и idf рассчитывается следующим образом:
$$
idf(t)= \log{\frac{1+n_d}{1+df(d,t)}}+1
$$
где n_d - это количество документов; df(d,t) - документная частота слова t